In [9]:
import pandas as pd
from collections import defaultdict
import re

EXCEL_PATH = "ORS (2015_2017_2019) copied 2025-7-5.xlsx"
SHEET_NAME = "HIV2015"  

In [10]:
def to_num(x):
    return pd.to_numeric(pd.Series([x]), errors="coerce").iloc[0]


def impact_piece(daly, cov, x):
    if pd.isna(daly) or pd.isna(cov) or pd.isna(x):
        return 0.0
    prod = float(cov) * float(x)
    denom = 1.0 - prod
    if denom <= 0:
        return 0.0
    return float(daly) * prod / denom

In [11]:
def build_country_df(df_raw):
    header_row = 2
    country_start_row = 4

    headers = []
    seen = {}

    for c in range(df_raw.shape[1]):
        h = df_raw.iat[header_row, c]
        if pd.isna(h):
            h = f"Unnamed_{c}"
        else:
            h = str(h).strip()

        if h in seen:
            seen[h] += 1
            h = f"{h}_{seen[h]}"
        else:
            seen[h] = 1
        headers.append(h)

    tmp = df_raw.iloc[country_start_row:].copy()
    tmp.columns = headers

    exclude_exact = {
        "Country", "WHO Group", "Children (0-14)", "Adults (15+)", "All ages", "Year",
        "WHO groupings (A & B):", "Regimen breakdowns", "Sources / More details",
        "Drugs and manufacturers", "Antiretroviral coverage:"
    }

    def is_real_country_row(row):
        name = row.get("Country", None)
        daly = row.get("DALY", None)

        if pd.isna(name) or not isinstance(name, str):
            return False
        s = name.strip()
        if not s:
            return False

        if s in exclude_exact:
            return False
        if s.lower().startswith("http"):
            return False
        if ":" in s:
            return False
        if "impact" in s.lower():
            return False
        if "coverage" in s.lower():
            return False
        if "regimen" in s.lower():
            return False

        daly_num = to_num(daly)
        if pd.isna(daly_num):
            return False

        return True

    country_df = tmp[tmp.apply(is_real_country_row, axis=1)].copy().reset_index(drop=True)
    return country_df

In [12]:
def detect_drug_cols(country_df):
    start_col = "# Children Prevalent Cases Receiving Treatment"
    end_col = "Overall Treatment Impact"
    start_idx = country_df.columns.get_loc(start_col) + 1
    end_idx = country_df.columns.get_loc(end_col)
    return country_df.columns[start_idx:end_idx].tolist()

In [13]:
def parse_regimen_df(df_raw, drug_cols):
    regimen_header_row = 3
    c_regimen = 44
    c_adult_prop = 45
    c_adult_eff = 46
    c_child_prop = 47
    c_child_eff = 48
    c_num_drugs = 49
    c_adult_x = 50
    c_child_x = 51

    records = []
    for r in range(regimen_header_row + 1, df_raw.shape[0]):
        regimen_val = df_raw.iat[r, c_regimen]
        if pd.isna(regimen_val):
            continue

        records.append({
            "row_idx": r,
            "regimen": str(regimen_val).strip(),
            "adult_proportion": df_raw.iat[r, c_adult_prop],
            "adult_efficacy": df_raw.iat[r, c_adult_eff],
            "children_proportion": df_raw.iat[r, c_child_prop],
            "children_efficacy": df_raw.iat[r, c_child_eff],
            "number_of_drugs": df_raw.iat[r, c_num_drugs],
            "adult_x": df_raw.iat[r, c_adult_x],
            "child_x": df_raw.iat[r, c_child_x],
        })

    regimen_df = pd.DataFrame(records)

    for col in [
        "adult_proportion", "adult_efficacy",
        "children_proportion", "children_efficacy",
        "number_of_drugs", "adult_x", "child_x"
    ]:
        regimen_df[col] = pd.to_numeric(regimen_df[col], errors="coerce")

    def looks_like_real_regimen(name):
        if pd.isna(name):
            return False
        s = str(name).strip()
        if not s:
            return False
        s_lower = s.lower()
        exclude_contains = [
            "regimens",
            "determination",
            "original data point",
            "average of all data points",
            "quadrant"
        ]
        return not any(x in s_lower for x in exclude_contains)

    regimen_df = regimen_df[regimen_df["regimen"].apply(looks_like_real_regimen)].copy()
    regimen_df = regimen_df[
        regimen_df[["adult_proportion", "children_proportion", "adult_x", "child_x"]]
        .notna()
        .any(axis=1)
    ].copy()

    def parse_regimen_drugs(regimen_name):
        s = str(regimen_name).strip()
        if s.lower().startswith("others"):
            return []
        parts = [p.strip() for p in s.split("+")]
        return [re.sub(r"\s+", " ", p) for p in parts if p.strip()]

    regimen_df["parsed_drugs"] = regimen_df["regimen"].apply(parse_regimen_drugs)
    regimen_df["is_special_bucket"] = regimen_df["regimen"].str.lower().str.startswith("others")

    # keep only parsed drugs relevant to final output
    drug_set = set(drug_cols)
    regimen_df["parsed_drugs"] = regimen_df["parsed_drugs"].apply(
        lambda lst: [d for d in lst if d in drug_set]
    )

    return regimen_df.reset_index(drop=True)

In [14]:
def is_no_data_coverage_case(country_row):
    cleaned_cov = to_num(country_row.get("Cleaned coverage", None))
    reported = country_row.get("Reported number of people receiving antiretroviral therapy", None)
    est_cov_text = country_row.get(
        "Estimated antiretroviral therapy coverage among people living with HIV (%)", None
    )

    reported_no_data = isinstance(reported, str) and reported.strip().lower() == "no data"
    high_income_text = isinstance(est_cov_text, str) and "high-income country" in est_cov_text.lower()

    return pd.isna(cleaned_cov) and (reported_no_data or high_income_text)


In [15]:

def compute_country_impacts(country_row, regimen_df, drug_cols):
    if is_no_data_coverage_case(country_row):
        return {d: 0.0 for d in drug_cols}

    adult_daly = to_num(country_row["Adult DALYs"])
    child_daly = to_num(country_row["Children DALYs"])
    adult_cov = to_num(country_row["% Treatment Coverage_2"])
    child_cov = to_num(country_row["% Treatment Coverage_3"])
    retention_all = to_num(country_row["Retention Rate"])

    pre_norm = defaultdict(float)

    for _, rr in regimen_df.iterrows():
        regimen_name = str(rr["regimen"]).strip()
        n_drugs = rr["number_of_drugs"]
        ax = rr["adult_x"]
        cx = rr["child_x"]

        adult_imp = impact_piece(adult_daly, adult_cov, ax)
        child_imp = impact_piece(child_daly, child_cov, cx)
        regimen_total = adult_imp + child_imp

        if not bool(rr["is_special_bucket"]):
            if pd.isna(n_drugs) or n_drugs == 0:
                continue
            per_drug = regimen_total / float(n_drugs)
            for d in rr["parsed_drugs"]:
                pre_norm[d] += per_drug
            continue

        lower = regimen_name.lower()
        if "others tdf based" in lower:
            pre_norm["TDF"] += regimen_total
        elif lower == "others":
            pass

    norm_factor = 1.0 if pd.isna(retention_all) else (100.0 - float(retention_all)) / 100.0
    return {d: pre_norm.get(d, 0.0) * norm_factor for d in drug_cols}


In [ ]:
df_raw = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_NAME, header=None)

country_df = build_country_df(df_raw)
drug_cols = detect_drug_cols(country_df)
regimen_df = parse_regimen_df(df_raw, drug_cols)

computed_records = []
for _, crow in country_df.iterrows():
    country_name = str(crow["Country"]).strip()
    impacts = compute_country_impacts(crow, regimen_df, drug_cols)
    rec = {"Country": country_name, **impacts}
    computed_records.append(rec)

computed_wide = pd.DataFrame(computed_records)[["Country"] + drug_cols]

computed_long = computed_wide.melt(
    id_vars=["Country"],
    value_vars=drug_cols,
    var_name="Drug",
    value_name="Impact Score"
).copy()

computed_long.to_csv("impact_score_narrow_computed.csv", index=False)
computed_wide.to_csv("impact_score.csv", index=False)

sheet_wide = country_df[["Country"] + drug_cols].copy()
for d in drug_cols:
    sheet_wide[d] = pd.to_numeric(sheet_wide[d], errors="coerce")

compare_wide = sheet_wide.merge(computed_wide, on="Country", suffixes=("_sheet", "_computed"))

diff_records = []
for _, row in compare_wide.iterrows():
    country = row["Country"]
    for d in drug_cols:
        sheet_val = row[f"{d}_sheet"]
        comp_val = row[f"{d}_computed"]
        diff = sheet_val - comp_val
        diff_records.append({
            "Country": country,
            "Drug": d,
            "Sheet Impact": sheet_val,
            "Computed Impact": comp_val,
            "Difference": diff,
            "Abs Difference": abs(diff) if pd.notna(diff) else None
        })

pd.DataFrame(diff_records).to_csv("impact_score_comparison_vs_sheet.csv", index=False)